<a href="https://colab.research.google.com/github/yukinaga/minnano_rl/blob/main/section_2/01_simple_reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# シンプルな強化学習の実装
強化学習の一種である、Q学習を実装します。  
今回は、Q学習により重力下で飛行する物体の制御を行います。

## ライブラリの導入
数値計算のためにNumPy、グラフ表示のためにmatplotlibを導入します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import animation, rc

## Brainクラス
エージェントの頭脳となるクラスです。  
Q-Tableを使用し、より多くの報酬が得られるようにQ値を調整します。  
Q値の更新量は以下の式で表されます。  
  
`Q値の更新量 = 学習係数 x ( 報酬 + 割引率 x 次の状態で最大のQ値 - 現在のQ値 )  `
  
また、ある状態における行動を決定する`get_action`メソッドでは、ε-greedy法により行動が選択されます。  
学習の初期ではランダムに行動が決定されますが、学習が進むとQ値の高い行動が選択されるようになります。

In [ ]:
class Brain:
    def __init__(self, n_state, w_y, w_vy, n_action, gamma=0.9, r=0.99, lr=0.01):
        self.n_state = n_state  # 状態の数
        self.w_y = w_y  # 位置の刻み幅
        self.w_vy = w_vy  # 速度の刻み幅
        self.n_action = n_action  # 行動の数

        self.eps = 1.0  # ε
        self.gamma = gamma  # 割引率
        self.r = r  # εの減衰率
        self.lr = lr  # 学習係数

        self.q_table = np.random.rand(n_state*n_state, n_action)  # Q-Table

    def quantize(self, state, n_state, w):  # 状態の値を整数のインデックスに変換
        min = - n_state / 2 * w
        nw = (state - min) / w
        nw = int(nw)
        nw = 0 if nw < 0 else nw
        nw = n_state-1 if nw >= n_state-1 else nw
        return nw

    def train(self, states, next_states, action, reward, terminal):  # Q-Tableを訓練
        i = self.quantize(states[0], self.n_state, self.w_y)  # 位置のインデックス
        j = self.quantize(states[1], self.n_state, self.w_vy)  # 速度のインデックス
        q = self.q_table[i*self.n_state+j, action]  # 現在のQ値

        next_i = self.quantize(next_states[0], self.n_state, self.w_y)  # 次の位置のインデックス
        next_j = self.quantize(next_states[1], self.n_state, self.w_vy)  # 次の速度のインデックス
        q_next = np.max(self.q_table[next_i*self.n_state+next_j])  # 次の状態で最大のQ値

        if terminal:
            self.q_table[i*self.n_state+j, action] = q + self.lr*reward  # 終了時は報酬のみ使用
        else:
            self.q_table[i*self.n_state+j, action] = q + self.lr*(reward + self.gamma*q_next - q)  # Q値の更新式

    def get_action(self, states):
        if np.random.rand() < self.eps:  # ランダムな行動
            action = np.random.randint(self.n_action)
        else:  # Q値の高い行動を選択
            i = self.quantize(states[0], self.n_state, self.w_y)
            j = self.quantize(states[1], self.n_state, self.w_vy)
            action = np.argmax(self.q_table[i*self.n_state+j])
        if self.eps > 0.1:  # εの下限
            self.eps *= self.r
        return action

## エージェントのクラス
エージェントをクラスとして実装します。  
x座標が-1から1まで、y座標が-1から1までの正方形の領域を考えますが、エージェントの初期位置は左端中央とします。  
そして、エージェントが右端に達した際は報酬として1を与え、終了とします。  
また、エージェントが上端もしくは下端に達した際は報酬として-1を与え、終了とします。  
  
x軸方向には等速度で移動します。  
行動には、自由落下とジャンプの2種類があります。自由落下の場合は重量加速度をy速度に加えます。ジャンプの場合は、y速度を予め設定した値に変更します。

In [ ]:
class Agent:
    def __init__(self, v_x, v_y_sigma, v_jump, brain):
        self.v_x = v_x  # x速度
        self.v_y_sigma = v_y_sigma  # y速度、初期値の標準偏差
        self.v_jump = v_jump  # ジャンプ速度
        self.brain = brain
        self.reset()

    def reset(self):
        self.x = -1  # 初期x座標
        self.y = 0  # 初期y座標
        self.v_y = self.v_y_sigma * np.random.randn()  # 初期y速度

    def step(self, g):  # 時間を1つ進める g:重力加速度
        states = np.array([self.y, self.v_y])
        self.x += self.v_x
        self.y += self.v_y

        reward = 0  # 報酬
        terminal = False  # 終了判定
        if self.x>1.0:
            reward = 1
            terminal = True
        elif self.y<-1.0 or self.y>1.0:
            reward = -1
            terminal = True
        reward = np.array([reward])

        action = self.brain.get_action(states)
        if action == 0:
            self.v_y -= g   # 自由落下
        else:
            self.v_y = self.v_jump  # ジャンプ
        next_states = np.array([self.y, self.v_y])
        self.brain.train(states, next_states, action, reward, terminal)

        if terminal:
            self.reset()

## 環境のクラス
環境をクラスとして実装します。  
このクラスの役割は、重力加速度を設定し、時間を前に進めるのみです。

In [ ]:
class Environment:
    def __init__(self, agent, g):
        self.agent = agent
        self.g = g  # 重力加速度

    def step(self):
        self.agent.step(self.g)
        return (self.agent.x, self.agent.y)

## アニメーション
今回は、matplotlibを使ってエージェントの飛行をアニメーションで表します。  
アニメーションには、matplotlib.animationのFuncAnimation関数を使用します。  

In [ ]:
def animate(environment, interval, frames):
    fig, ax = plt.subplots()
    plt.close()
    ax.set_xlim(( -1, 1))
    ax.set_ylim((-1, 1))
    sc = ax.scatter([], [])

    def plot(data):
        x, y = environment.step()
        sc.set_offsets(np.array([[x, y]]))
        return (sc,)

    return animation.FuncAnimation(fig, plot, interval=interval, frames=frames, blit=True)

## ランダムな行動
まずは、エージェントがランダムに行動する例をみていきましょう。  
`r`の値を1に設定し、εが減衰しないようにすることで、エージェントは完全にランダムな行動を選択するようになります。

In [ ]:
n_state = 10
w_y = 0.2
w_vy = 0.2
n_action = 2
brain = Brain(n_state, w_y, w_vy, n_action, r=1.0)  # εが減衰しない

v_x = 0.05
v_y_sigma = 0.1
v_jump = 0.2
agent = Agent(v_x, v_y_sigma, v_jump, brain)

g = 0.2
environment = Environment(agent, g)

anim = animate(environment, 50, 1024)
rc("animation", html="jshtml")
anim

運良く右端に到達することもありますが、大抵は上端もしくは下端にぶつかってしまいます。

## Q学習の導入
`r`の値を0.99に設定し、εが減衰するようにします。  
これにより、Q学習の結果が行動に反映されるようになります。

In [ ]:
n_state = 10
w_y = 0.2
w_vy = 0.2
n_action = 2
brain = Brain(n_state, w_y, w_vy, n_action, r=0.99)  # εが減衰する

v_x = 0.05
v_y_sigma = 0.1
v_jump = 0.2
agent = Agent(v_x, v_y_sigma, v_jump, brain)

g = 0.2
environment = Environment(agent, g)

anim = animate(environment, 50, 1024)
rc("animation", html="jshtml")
anim

学習が進むと、上下の端にぶつらずに右端まで飛べるようになります。